# Multimodal Mountain Peak Search — Colab Notebook

In this notebook we do the following:
- Install dependencies
- Connect to **Elasticsearch**
- Create indices
- Index a small **peaks catalog** (text+image blended vectors)
- Index a few **photos**
- Run **text → image** search and **identify-from-photo** search


In [23]:
#@title Configure Elasticsearch (API key recommended)
import os, base64

# Prompt for Elasticsearch URL:
ES_URL = input("Elasticsearch url(https://localhost:9200): ") or "https://localhost:9200"


#Prompt Elasticsearch base64 api key:
ES_API_KEY_B64 = input("Elasticsearch base64 api key")


print("ES_URL     :", ES_URL)
print("API_KEY_B64:", "set" if ES_API_KEY_B64 else "MISSING")

# Propagate for scripts
os.environ["ES_URL"] = ES_URL
os.environ["ES_API_KEY_B64"] = ES_API_KEY_B64

# Optional model override
# os.environ["SIGLIP_MODEL_ID"] = "google/siglip-so400m-patch14-384"


Elasticsearch url(https://localhost:9200): 
Elasticsearch base64 api keyw
ES_URL     : https://localhost:9200
API_KEY_B64: set


In [2]:
#@title Clone Github repo
REPO_URL = "https://github.com/navneet83/multimodal-mountain-peak-search"
TARGET_DIR = "/content/multimodal-mountain-peak-search"
import os, shutil, subprocess, sys

if os.path.exists(TARGET_DIR):
    shutil.rmtree(TARGET_DIR)

print("Cloning:", REPO_URL)
rc = subprocess.call(["git","clone","--depth","1", REPO_URL, TARGET_DIR])
if rc != 0:
    raise SystemExit("❌ Clone failed. Check the repo URL or network.")

os.chdir(TARGET_DIR)
sys.path.insert(0, os.path.join(TARGET_DIR, "src"))  # import ai_mpi.embeddings
print("Cloned and cwd set to", TARGET_DIR)

Cloning: https://github.com/navneet83/multimodal-mountain-peak-search
Cloned and cwd set to /content/multimodal-mountain-peak-search


In [ ]:
#@title Install dependencies
!pip -q install --upgrade pip
!pip install -r requirements.txt
print("Installed")

In [ ]:
#@title Create indices in Elasticsearch
!python scripts/create_indices.py --recreate || python scripts/create_indices.py
print("Indices ready")

In [ ]:
#@title Index peaks (blended text + reference images)
!python scripts/embed_and_index_photos.py --index-peaks --peaks-yaml data/peaks.yaml --peaks-images-root data/peaks --blend-alpha-text 0.55 --blend-max-images 3
print("Peaks indexed")

In [ ]:
#@title Index your photos
!python scripts/embed_and_index_photos.py --index-photos --images data/images --topk-predicted 5
print("✅ Photos indexed")

In [ ]:
#@title Text → image search (type a peak name)
query = "Pumori"  #@param ["Ama Dablam", "Pumori", "Mount Everest"] {allow-input: true}
k = 12  #@param {type:"slider", min:6, max:30, step:2}
num_candidates = 4000  #@param {type:"slider", min:1000, max:6000, step:1000}

!python scripts/query_by_peak_name.py --peak query --k {k} --num-candidates {num_candidates}

In [ ]:
#@title Identify from photo → similar photos (upload OR reuse a repo image)
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  !python scripts/identify_from_picture_find_similar_peaks.py --image "{fn}" --neighbors 30
